In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

def get_yfData(tickers, period, columns):
    yfDf = yf.download(tickers = [tickers], period = period)
    for column in range(len(columns)):
        yfDf[column].fillna(method = "ffill")
    return yfDf

df = yf.download(tickers=['^GSPC'], period='3y')
df['Close'].fillna(method = "ffill")
df.head(n=5)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-06-18,2906.709961,2930.790039,2905.439941,2917.750000,2917.750000,3437620000
2019-06-19,2920.550049,2931.739990,2911.429932,2926.459961,2926.459961,3287890000
2019-06-20,2949.600098,2958.060059,2931.500000,2954.179932,2954.179932,3905940000
2019-06-21,2952.709961,2964.149902,2946.870117,2950.459961,2950.459961,5000120000
2019-06-24,2951.419922,2954.919922,2944.050049,2945.350098,2945.350098,3136250000


In [9]:
timesteps = df.index.to_numpy()
prices = df['Close'].to_numpy().reshape(-1, 1)

test_size = int(0.8 * len(prices))

X_train, y_train = timesteps[:test_size], prices[test_size:]
X_test, y_test = timesteps[test_size:], prices[test_size:]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(606,) (152, 1) (152,) (152, 1)


In [13]:
def label_windows(x, horizon = 1):
    return x[:, :-horizon], x[:, -horizon:]

def make_windows(x, window_size = 7, horizon = 1):
    window_step = np.expand_dims(np.arange(window_size + horizon), axis = 0)
    window_indexes = window_step + np.expand_dims(np.arange(len(x) - (window_size + horizon - 1)), axis = 0).T
    windowed_array = x[window_indexes]
    windows, labels = label_windows(windowed_array)
    return windows, labels

full_windows, full_labels = make_windows(prices, window_size = 7, horizon = 1)
print(full_windows.shape, full_labels.shape)

def split_windows(windows, labels):
    test_split = int(0.8 * len(windows))
    train_windows, train_labels = windows[:test_split], labels[:test_split]
    test_windows, test_labels = windows[test_split:], labels[test_split:]
    return train_windows, train_labels, test_windows, test_labels

train_windows, train_labels, test_windows, test_labels = split_windows(full_windows, full_labels)

(751, 7, 1) (751, 1, 1)
